<a href="https://colab.research.google.com/github/felipe7leal/Quant-Algo/blob/main/Bitcoin_Tese_e_Estrategia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

In [ ]:
#install
!pip install plotly
!pip install ta
!pip install vectorbt

In [6]:
#import
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.momentum import RSIIndicator
import vectorbt as vbt

In [7]:
# Definindo parâmetros | Defining parameters
ticker = "BTC-USD"
start="2010-01-01"
period="1w"

# Baixando os dados históricos do preço do Bitcoin | Downloading the historical Bitcoin price data
data = yf.download(tickers=ticker, period=period, start=start)

[*********************100%%**********************]  1 of 1 completed


#● Análise quantitativa
#● Quantitative analysis

##Bitcoin é uma nova classe de ativo que nunca existiu na história da humanidade, pois combina três classes de ativos: dinheiro (transações), ações (investimentos de risco) e ouro (reserva de valor).
##Estamos em uma clara **tendência de alta** no longo prazo e é o ativo que melhor performou na década. Segundo um estudo recente realizado pela Hashdex em parceria com a Quantum, nos últimos dez anos, o BTC foi o ativo que mais se valorizou, registrando uma alta astronômica de 7.880%.
##O Bitcoin tem um grande potencial de valorização, dado que foi criado há pouco tempo e sua adoção está crescendo rapidamente, comparável à adoção da internet. Além disso, é deflacionário e visto como escasso devido à sua oferta imutável de 21 milhões de BTC.
##Um fator promissor para a valorização futura do Bitcoin é a recente regulamentação. Isso não só está atraindo o varejo, mas também o investimento institucional, como evidenciado pela resolução CVM 175 e a solicitação de um ETF spot junto à SEC pela BlackRock, a maior gestora do mundo.

--

##Bitcoin is a new asset class that has never existed in human history, as it combines three asset classes: money (for transactions), stocks (for risk investments), and gold (as a store of value).
##We are in a clear long-term uptrend, and it is the best-performing asset of the decade. According to a recent study by Hashdex in partnership with Quantum, BTC has been the best-performing asset over the past ten years, with an astronomical rise of 7,880%.
##Bitcoin has significant upside potential, given that it was created recently and its adoption is growing rapidly, comparable to the adoption of the internet. In addition, it is deflationary and seen as scarce due to its immutable supply of 21 million BTC.
##A promising factor for the future appreciation of Bitcoin is recent regulation. This is not only attracting retail investors, but also institutional investment, as evidenced by CVM Resolution 175 and BlackRock's (the world's largest asset manager) request for a spot ETF with the SEC.

In [8]:
# Calculando a média móvel de 200 períodos | Calculating the 200-day moving average
data['MA200'] = data['Close'].rolling(window=200).mean()

# Criando o primeiro gráfico com o preço do Bitcoin e a média móvel de 200 períodos | Creating the first chart with the Bitcoin price and the 200-day moving average
fig1 = go.Figure(data=[
    go.Candlestick(x=data.index,
                   open=data['Open'],
                   high=data['High'],
                   low=data['Low'],
                   close=data['Close'],
                   name='Bitcoin'),
    go.Scatter(x=data.index, y=data['MA200'], mode='lines', name='MA200')
],
layout=go.Layout(yaxis_type='log', template='plotly_dark', xaxis_rangeslider_visible=False))
fig1.show()

## Bitcoin é considerado um dos ativos mais voláteis do mundo, com volatilidade anualizada de 80%. Para efeito de comparação, a volatilidade anualizada do S&P 500 é de 15%.
##Essa volatilidade pode ser um risco para os investidores, mas também pode ser uma oportunidade para os traders.

--

##Bitcoin is considered one of the most volatile assets in the world, with an annualized volatility of 80%. For comparison, the annualized volatility of the S&P 500 is 15%.
##This volatility can be a risk for investors, but it can also be an opportunity for traders.

In [9]:
# Calculando a volatilidade histórica | Calculating the historical volatility
data['returns'] = np.log(data['Close'] / data['Close'].shift(1))
data['volatility'] = data['returns'].rolling(window=252).std() * np.sqrt(252)

# Criando o segundo gráfico com o preço do Bitcoin e a volatilidade histórica | Creating the second chart with the Bitcoin price and the historical volatility
fig2 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig2.add_trace(go.Candlestick(x=data.index,
                   open=data['Open'],
                   high=data['High'],
                   low=data['Low'],
                   close=data['Close'],
                   name='Bitcoin'), row=1, col=1)
fig2.add_trace(go.Scatter(x=data.index, y=data['volatility'], mode='lines', name='Volatility'), row=2, col=1)
fig2.update_layout(yaxis_type='log', template='plotly_dark', xaxis_rangeslider_visible=False)
fig2.show()

#● Estratégia de investimento
#● Investment strategy


##Baseado no Halving e nas estações cripto, o ideal é adotar uma estratégia de position trade, visando surfar a assimetria do ativo nas estações primavera/verão e fazendo hedge da posição nas estações de outono e principalmente ‘inverno cripto’, onde ocorrem os maiores drawdowns.
##Existem diversas estratégias sofisticadas, utilizando análise on-chain para investimentos de longo prazo e de sentimento para trades de curto prazo. Entretanto, com uma estratégia simples e pragmática de análise técnica utilizando o RSI semanal ou a famosa Bull Market Support Band, podemos obter bons retornos/rentabilidade.

--

##Based on the Halving and crypto seasons, the ideal is to adopt a position trading strategy, aiming to surf the asset's asymmetry in the spring/summer seasons and hedging the position in the fall and especially the "crypto winter", where the biggest drawdowns occur.
##There are many sophisticated strategies, using on-chain analysis for long-term investments and sentiment for short-term trades. However, with a simple and pragmatic technical analysis strategy using the weekly RSI or the famous Bull Market Support Band, we can obtain good returns/performance.

In [10]:
# Calculando o RSI | Calculating the Relative Strength Index (RSI)
rsi = RSIIndicator(close=data['Close'], window=14)
data['RSI'] = rsi.rsi()

# Calculando as médias móveis de 20 e 21 semanas | Calculating the 20 and 21 week moving averages
sma = data['Close'].rolling(window=20 * 7).mean()
ema = data['Close'].ewm(span=21 * 7).mean()
data['SMA'] = sma
data['EMA'] = ema

# Criando o terceiro gráfico com o preço do Bitcoin, o RSI e as médias móveis do Bull Market Support Band | Creating the third chart with the Bitcoin price, the RSI, and the 20 and 21 week moving averages
fig3 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig3.add_trace(go.Candlestick(x=data.index,
                   open=data['Open'],
                   high=data['High'],
                   low=data['Low'],
                   close=data['Close'],
                   name='Bitcoin'), row=1, col=1)
fig3.add_trace(go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI'), row=2, col=1)
fig3.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA'), row=1, col=1)
fig3.add_trace(go.Scatter(x=data.index, y=data['EMA'], mode='lines', name='EMA'), row=1, col=1)
fig3.add_hline(y=57)
fig3.update_layout(yaxis_type='log', template='plotly_dark', xaxis_rangeslider_visible=False)
fig3.show()

#●  Backtest otimizado
#●  Backtesting & Optimization


In [11]:
# Baixando os dados de preço | Downloading price data
price = vbt.YFData.download(symbols="BTC-USD",interval="1d",start=start).get('Close')

##  Bull Market Support Band


In [12]:
# Calculando as médias móveis no time frame semanal | Calculating the moving averages on a weekly time frame
sma = price.rolling(window=20 * 7).mean()
ema = price.ewm(span=21 * 7).mean()

# Criando uma matriz de sinais de compra e venda com base nas médias móveis | Creating a matrix of buy and sell signals based on the moving averages
entries = price > sma
exits = price < ema

# Criando um portfólio com base nos sinais de compra e venda | Creating a portfolio based on the buy and sell signals
pf = vbt.Portfolio.from_signals(price, entries, exits)

In [13]:
# Exibindo o gráfico do portfólio | Plotting the portfolio's performance metrics
pf.plot().show()

In [14]:
# Exibindo as estatísticas do portfólio | Printing the portfolio's performance metrics
pf.stats()

Start                         2014-09-17 00:00:00+00:00
End                           2023-09-05 00:00:00+00:00
Period                               3276 days 00:00:00
Start Value                                       100.0
End Value                                  25056.200296
Total Return [%]                           24956.200296
Benchmark Return [%]                        5501.051921
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              61.819733
Max Drawdown Duration                 556 days 00:00:00
Total Trades                                         14
Total Closed Trades                                  14
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  42.857143
Best Trade [%]                              3575.449365
Worst Trade [%]                              -15

## Grid Search

Backtest de uma estratégia de negociação de reversão usando busca em grade, considerando o índice Sharpe para avaliar o desempenho ajustado ao risco do portfólio resultante para cada valor do RSI testado.

Backtest of a reversal trading strategy using grid search to find the RSI value that provides the best risk-adjusted return on the portfolio.


In [15]:
# Definindo o intervalo de valores para o RSI | Defining the range of RSI values to test
rsi_values = range(40, 61)

# Inicializando variáveis para armazenar os resultados | Initializing variables to store the results
best_rsi_value = None
best_sharpe_ratio = -np.inf

# Testando cada valor do RSI no intervalo definido | Testing each RSI value in the defined range
for rsi_value in rsi_values:
    # Calculando o RSI no time frame semanal | Calculating the RSI on a weekly time frame
    rsi = vbt.RSI.run(price, window=14 * 7).rsi

    # Criando uma matriz de sinais de compra e venda com base nos valores do RSI | Creating a matrix of buy and sell signals based on the RSI values
    entries = rsi > rsi_value
    exits = rsi < rsi_value

    # Criando um portfólio com base nos sinais de compra e venda | Creating a portfolio based on the buy and sell signal
    pf = vbt.Portfolio.from_signals(price, entries, exits)

    # Calculando o índice Sharpe do portfólio | Calculating the Sharpe ratio of the portfolio
    sharpe_ratio = pf.sharpe_ratio()

    # Verificando se o índice Sharpe é melhor do que o melhor índice Sharpe encontrado até agora | Checking if the Sharpe ratio is better than the best Sharpe ratio found so far
    if sharpe_ratio > best_sharpe_ratio:
        best_sharpe_ratio = sharpe_ratio
        best_rsi_value = rsi_value

# Exibindo o melhor valor do RSI encontrado | Printing the best RSI value found
print(f"O melhor valor do RSI encontrado foi: {best_rsi_value}")

# Calculando o RSI no time frame semanal com o melhor valor do RSI encontrado | Calculating the RSI on a weekly time frame with the best RSI value found
rsi = vbt.RSI.run(price, window=14 * 7).rsi

# Criando uma matriz de sinais de compra e venda com base nos valores do RSI | Creating a matrix of buy and sell signals based on the RSI values
entries = rsi > best_rsi_value
exits = rsi < best_rsi_value

# Criando um portfólio com base nos sinais de compra e venda | Creating a portfolio based on the buy and sell signals
pf = vbt.Portfolio.from_signals(price, entries, exits)


O melhor valor do RSI encontrado foi: 57


In [16]:
# Exibindo o gráfico do portfólio | Plotting the portfolio's performance metrics
pf.plot().show()

In [17]:
# Exibindo as estatísticas do portfólio | Printing the portfolio's performance metrics
print(pf.stats())

Start                         2014-09-17 00:00:00+00:00
End                           2023-09-05 00:00:00+00:00
Period                               3276 days 00:00:00
Start Value                                       100.0
End Value                                  13766.720182
Total Return [%]                           13666.720182
Benchmark Return [%]                        5501.051921
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              41.066502
Max Drawdown Duration                 525 days 00:00:00
Total Trades                                         43
Total Closed Trades                                  43
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  46.511628
Best Trade [%]                               335.000085
Worst Trade [%]                              -16

## A estratégia de RSI semanal tem um desempenho ligeiramente melhor do que a estratégia de Bull Market Support Band em termos de win rate (46,51% vs 42,86%), índice Sharpe (1,458514 vs 1,408211) e drawdown máxima (41,07% vs 61,82%). No entanto, a estratégia de Bull Market Support Band tem um retorno total maior (24956,20% vs 13666,72%) e um profit factor maior (6,74 vs 5,29).
##Isso sugere que a estratégia de RSI semanal pode ser uma boa opção para investidores que estão procurando uma estratégia ativa com um risco menor. A estratégia de Bull Market Support Band pode ser uma boa opção para investidores que estão procurando uma estratégia mais seletiva com um potencial de retorno maior.
##***É importante notar que os resultados passados não garantem resultados futuros. Os investidores devem realizar sua própria pesquisa e consultar um consultor financeiro antes de tomar qualquer decisão de investimento.***
--
##The weekly RSI strategy has a slightly better performance than the Bull Market Support Band strategy in terms of win rate (46.51% vs 42.86%), Sharpe ratio (1.458514 vs 1.408211), and maximum drawdown (41.07% vs 61.82%). However, the Bull Market Support Band strategy has a higher total return (24956.20% vs 13666.72%) and a higher profit factor (6.74 vs 5.29).
##This suggests that the weekly RSI strategy may be a good option for investors looking for an active strategy with lower risk. The Bull Market Support Band strategy may be a good option for investors looking for a more selective strategy with greater potential for return.
##***It is important to note that past performance does not guarantee future results. Investors should do their own research and consult a financial advisor before making any investment decision.***